In [4]:
import numpy as np
import pandas as pd
import os
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
os.environ["MLFLOW_TRACKING_URI"] = "http://127.0.0.1:5000"
mlflow.set_experiment("Data Poisoning Attack")

2025/11/15 06:11:03 INFO mlflow.tracking.fluent: Experiment with name 'Data Poisoning Attack' does not exist. Creating a new experiment.


<Experiment: artifact_location='gs://mlops-week-8/1', creation_time=1763187063158, experiment_id='1', last_update_time=1763187063158, lifecycle_stage='active', name='Data Poisoning Attack', tags={}>

In [5]:
# Load the raw iris data
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Series(iris.target, name="target")

# Create a clean, un-poisoned test set.
# We MUST test our models against clean data.
X_train_clean, X_test, y_train_clean, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Total training set size: {len(X_train_clean)}")
print(f"Clean test set size: {len(X_test)}")

Total training set size: 120
Clean test set size: 30


In [6]:
# --- 3. Define the Poisoning Function ---
def poison_data(X_train, y_train, poison_percentage):
    """
    Poisons a percentage of the training data by replacing
    features with random, out-of-distribution numbers.
    """
    if poison_percentage == 0:
        return X_train, y_train
        
    # Combine X and y for easy sampling
    train_data = pd.concat([X_train, y_train], axis=1)
    
    # Calculate number of rows to poison
    n_poison = int(len(train_data) * poison_percentage)
    print(f"Poisoning {n_poison} rows...")
    
    # Get the indices of the rows to poison
    poison_indices = np.random.choice(train_data.index, n_poison, replace=False)
    
    # Create the poisoned data
    # We use random numbers from 50-100, which is far
    # outside the normal Iris data range (0-10).
    poison_values = np.random.uniform(50, 100, size=(n_poison, X_train.shape[1]))
    
    # Create copies to avoid modifying the original data
    X_train_poisoned = X_train.copy()
    y_train_poisoned = y_train.copy()

    # Apply the poison
    # We replace the *features* but keep the *label*.
    # This teaches the model a wrong pattern.
    X_train_poisoned.loc[poison_indices] = poison_values
    
    return X_train_poisoned, y_train_poisoned

In [7]:
poison_levels = [0.0, 0.05, 0.10, 0.50]

for percent in poison_levels:
    run_name = f"run_poison_{int(percent*100)}pct"
    
    with mlflow.start_run(run_name=run_name) as run:
        print(f"\n--- Starting Run: {run_name} ---")
        
        # 1. Log the poisoning parameter
        mlflow.log_param("poison_percentage", percent)
        
        # 2. Create the poisoned dataset
        X_train_poisoned, y_train_poisoned = poison_data(
            X_train_clean, y_train_clean, percent
        )
        
        # 3. Train the model on the (potentially) poisoned data
        model = LogisticRegression(max_iter=200)
        model.fit(X_train_poisoned, y_train_poisoned)
        
        # 4. Evaluate the model on the CLEAN test set
        preds = model.predict(X_test)
        accuracy = accuracy_score(y_test, preds)
        
        print(f"Test Accuracy on CLEAN data: {accuracy:.4f}")
        
        # 5. Log the results
        mlflow.log_metric("clean_test_accuracy", accuracy)
        mlflow.sklearn.log_model(model, "model")

print("\n--- All experiments complete. ---")


--- Starting Run: run_poison_0pct ---


2025/11/15 06:14:05 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Test Accuracy on CLEAN data: 0.9667


2025/11/15 06:14:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/15 06:14:10 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run run_poison_0pct at: http://127.0.0.1:5000/#/experiments/1/runs/e2fa74ba7a4046a7bbd39af527545f9c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1

--- Starting Run: run_poison_5pct ---
Poisoning 6 rows...
Test Accuracy on CLEAN data: 0.7333


...

🏃 View run run_poison_5pct at: http://127.0.0.1:5000/#/experiments/1/runs/7c3d0c5cc1f9449eb5424abad9fb3cb6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1

--- Starting Run: run_poison_10pct ---
Poisoning 12 rows...
Test Accuracy on CLEAN data: 0.7667


2025/11/15 06:14:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/15 06:14:17 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run run_poison_10pct at: http://127.0.0.1:5000/#/experiments/1/runs/a00723179e7145819cfc6ba84329da4f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1

--- Starting Run: run_poison_50pct ---
Poisoning 60 rows...
Test Accuracy on CLEAN data: 0.3333


2025/11/15 06:14:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run run_poison_50pct at: http://127.0.0.1:5000/#/experiments/1/runs/c43f7aaf800f4108af284315b6cad26f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1

--- All experiments complete. ---
